# Лабораторная работа 3: Классификация и нейронные сети

## Введение

Целью данной лабораторной работы является изучение методов классификации данных, реализованных в библиотеке Scikit-Learn, а также ознакомление с нейронными сетями с использованием библиотек TensorFlow и TensorBoard. Классификация представляет собой одну из основных задач машинного обучения, направленную на отнесение объектов к заранее определенным классам на основе их признаков.

В рамках работы необходимо выбрать и подготовить датасет для классификации, затем построить классификационные модели с помощью пяти различных методов: наивный Байесовский классификатор, деревья решений, линейный дискриминантный анализ, метод опорных векторов и метод ближайших соседей. Каждый метод имеет свои особенности и области применения, что позволяет провести сравнительный анализ их эффективности на конкретном датасете.

Дополнительно в работе реализуется и тестируется нейронная сеть на TensorFlow, исследуется эффект настройки гиперпараметров и визуализируется процесс обучения с помощью инструмента TensorBoard. Нейронные сети представляют собой мощный инструмент машинного обучения, способный моделировать сложные нелинейные зависимости между признаками и целевой переменной.

Работа направлена на приобретение практических навыков работы с различными методами классификации, понимание их принципов работы, умение оценивать качество моделей с помощью различных метрик и настраивать гиперпараметры для улучшения результатов.


In [ ]:
# Импорт необходимых библиотек
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.naive_bayes import GaussianNB, MultinomialNB, ComplementNB, BernoulliNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import (accuracy_score, precision_score, recall_score, 
                            f1_score, roc_auc_score, classification_report, 
                            confusion_matrix, roc_curve, auc)
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

# Настройки
DATA_PATH = 'Iris.csv'
TEST_SIZE = 0.2
RANDOM_STATE = 42
CV_FOLDS = 5
EPOCHS = 100
BATCH_SIZE = 32
VALIDATION_SPLIT = 0.2
TENSORBOARD_DIR = 'logs'
FIG_SIZE = (12, 8)
DPI = 100

# Создание папок для графиков и логов
photos_dir = os.path.join('photos')
os.makedirs(photos_dir, exist_ok=True)
os.makedirs(TENSORBOARD_DIR, exist_ok=True)

# Настройка визуализации
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = FIG_SIZE
plt.rcParams['figure.dpi'] = DPI

# Фиксация seed для воспроизводимости
np.random.seed(RANDOM_STATE)
tf.random.set_seed(RANDOM_STATE)


## Выбор и подготовка датасета

В данной работе используется датасет Iris, который является классическим набором данных для задач классификации. Датасет содержит информацию о трех видах цветов ириса (Iris setosa, Iris versicolor, Iris virginica) и включает четыре признака: длина чашелистика, ширина чашелистика, длина лепестка и ширина лепестка. Датасет является сбалансированным, содержащим по 50 образцов каждого класса, что делает его идеальным для изучения методов классификации.


In [ ]:
# Загрузка и анализ данных
df = pd.read_csv(DATA_PATH)
print("Форма данных:", df.shape)
print("\nПервые строки:")
df.head()


In [ ]:
# Информация о данных
df.info()


In [ ]:
# Распределение классов
print("Распределение классов:")
print(df['Species'].value_counts())

# Описательная статистика
print("\nОписательная статистика:")
df.describe()


In [ ]:
# Визуализация распределения признаков
fig, axes = plt.subplots(2, 2, figsize=(14, 10))
axes = axes.flatten()

feature_names = ['SepalLengthCm', 'SepalWidthCm', 'PetalLengthCm', 'PetalWidthCm']
for i, feature in enumerate(feature_names):
    for species in df['Species'].unique():
        subset = df[df['Species'] == species]
        axes[i].hist(subset[feature], alpha=0.6, label=species, bins=20)
    axes[i].set_title(f'Распределение {feature}')
    axes[i].set_xlabel(feature)
    axes[i].set_ylabel('Частота')
    axes[i].legend()
    axes[i].grid(True, alpha=0.3)

plt.tight_layout()
photos_path = os.path.join(photos_dir, 'feature_distributions.png')
plt.savefig(photos_path, dpi=DPI, bbox_inches='tight')
plt.show()


In [ ]:
# Предобработка данных
# Удаление ID если есть
if 'Id' in df.columns:
    df = df.drop('Id', axis=1)

# Разделение на признаки и целевую переменную
X = df.drop('Species', axis=1)
y = df['Species']

# Кодирование меток
le = LabelEncoder()
y_encoded = le.fit_transform(y)

# Стандартизация
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

print(f"Признаки: {list(X.columns)}")
print(f"Классы: {le.classes_}")
print(f"Размерность данных: {X_scaled.shape}")


## Разбиение выборки

Для оценки качества моделей классификации данные были разделены на обучающую и тестовую выборки в соотношении 80/20. Разделение было выполнено с использованием стратификации по классам, что обеспечивает сохранение пропорций классов в обеих выборках и позволяет получить более надежную оценку качества моделей.


In [ ]:
# Разделение на train/test
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y_encoded, test_size=TEST_SIZE, random_state=RANDOM_STATE, stratify=y_encoded
)

print(f"Размер обучающей выборки: {X_train.shape[0]} ({X_train.shape[0]/len(X_scaled)*100:.1f}%)")
print(f"Размер тестовой выборки: {X_test.shape[0]} ({X_test.shape[0]/len(X_scaled)*100:.1f}%)")
print(f"Количество признаков: {X_train.shape[1]}")


## Методы классификации

В данной работе были реализованы и протестированы пять различных методов классификации, каждый из которых имеет свои особенности и области применения. Наивный Байесовский классификатор основан на теореме Байеса и предполагает независимость признаков при заданном классе. В работе использовались четыре варианта наивного Байеса: GaussianNB для непрерывных признаков, MultinomialNB и ComplementNB для дискретных признаков, и BernoulliNB для бинарных признаков.

Деревья решений представляют собой непараметрический метод, который строит иерархическую структуру правил для классификации объектов. Деревья решений легко интерпретируемы и могут обрабатывать нелинейные зависимости, однако склонны к переобучению. Линейный дискриминантный анализ предполагает, что данные каждого класса имеют нормальное распределение с одинаковой ковариационной матрицей, и находит линейные границы между классами.

Метод опорных векторов (SVM) находит оптимальную разделяющую гиперплоскость в пространстве признаков, максимизируя зазор между классами. SVM может работать с различными ядерными функциями для обработки нелинейных зависимостей. Метод ближайших соседей (KNN) классифицирует объект на основе классов его k ближайших соседей в пространстве признаков. KNN является простым и интуитивно понятным методом, но может быть вычислительно затратным для больших наборов данных.


In [ ]:
# Функция для обучения классификаторов
def train_classifiers(X_train, X_test, y_train, y_test):
    """Обучение классификаторов"""
    classifiers = {
        'GaussianNB': GaussianNB(),
        'MultinomialNB': MultinomialNB(),
        'ComplementNB': ComplementNB(),
        'BernoulliNB': BernoulliNB(),
        'Decision Tree': DecisionTreeClassifier(random_state=RANDOM_STATE),
        'LDA': LinearDiscriminantAnalysis(),
        'SVM': SVC(random_state=RANDOM_STATE, probability=True),
        'KNN': KNeighborsClassifier()
    }
    
    results = {}
    
    for name, clf in classifiers.items():
        print(f"\nОбучение {name}...")
        
        # Обучение
        clf.fit(X_train, y_train)
        
        # Предсказания
        y_pred = clf.predict(X_test)
        y_pred_proba = clf.predict_proba(X_test) if hasattr(clf, 'predict_proba') else None
        
        # Метрики
        accuracy = accuracy_score(y_test, y_pred)
        precision = precision_score(y_test, y_pred, average='weighted')
        recall = recall_score(y_test, y_pred, average='weighted')
        f1 = f1_score(y_test, y_pred, average='weighted')
        
        # ROC-AUC (для многоклассовой задачи)
        if y_pred_proba is not None:
            try:
                roc_auc = roc_auc_score(y_test, y_pred_proba, multi_class='ovr', average='weighted')
            except:
                roc_auc = 0.0
        else:
            roc_auc = 0.0
        
        # Кросс-валидация
        cv_scores = cross_val_score(clf, X_train, y_train, cv=CV_FOLDS, scoring='accuracy')
        
        results[name] = {
            'classifier': clf,
            'accuracy': accuracy,
            'precision': precision,
            'recall': recall,
            'f1': f1,
            'roc_auc': roc_auc,
            'cv_mean': cv_scores.mean(),
            'cv_std': cv_scores.std(),
            'y_pred': y_pred,
            'y_pred_proba': y_pred_proba
        }
        
        print(f"  Accuracy: {accuracy:.4f}")
        print(f"  Precision: {precision:.4f}")
        print(f"  Recall: {recall:.4f}")
        print(f"  F1-Score: {f1:.4f}")
        print(f"  ROC-AUC: {roc_auc:.4f}")
        print(f"  CV Accuracy: {cv_scores.mean():.4f} (+/- {cv_scores.std()*2:.4f})")
    
    return results

# Обучение классификаторов
print("="*60)
print("ОБУЧЕНИЕ КЛАССИФИКАТОРОВ")
print("="*60)
results = train_classifiers(X_train, X_test, y_train, y_test)


In [ ]:
# Представление метрик в табличном виде
metrics_table = pd.DataFrame({
    'Метод': [],
    'Accuracy': [],
    'Precision': [],
    'Recall': [],
    'F1-Score': [],
    'ROC-AUC': []
})

for name, res in results.items():
    metrics_table = pd.concat([metrics_table, pd.DataFrame({
        'Метод': [name],
        'Accuracy': [res['accuracy']],
        'Precision': [res['precision']],
        'Recall': [res['recall']],
        'F1-Score': [res['f1']],
        'ROC-AUC': [res['roc_auc']]
    })], ignore_index=True)

print("Метрики качества классификаторов:")
metrics_table


## Настройка гиперпараметров

Настройка гиперпараметров является важным этапом в процессе построения моделей машинного обучения, так как правильный выбор параметров может значительно улучшить качество классификации. В данной работе была проведена настройка гиперпараметров для трех методов: деревьев решений, метода опорных векторов и метода ближайших соседей.

Для деревьев решений настраивались такие параметры, как максимальная глубина дерева и минимальное количество образцов для разделения узла. Максимальная глубина контролирует сложность модели и помогает предотвратить переобучение, в то время как минимальное количество образцов для разделения определяет, когда прекращается дальнейшее разделение узлов.

Для метода опорных векторов настраивались параметр регуляризации C, тип ядерной функции и параметр gamma. Параметр C контролирует компромисс между максимизацией зазора и минимизацией ошибки классификации, ядерная функция определяет тип разделяющей поверхности, а gamma контролирует влияние отдельных обучающих примеров.

Для метода ближайших соседей настраивались количество соседей k, тип весов (равномерные или взвешенные по расстоянию) и метрика расстояния. Количество соседей является критически важным параметром, так как слишком малое значение может привести к переобучению, а слишком большое - к недообучению.


In [ ]:
# Настройка гиперпараметров
print("\n" + "="*60)
print("НАСТРОЙКА ГИПЕРПАРАМЕТРОВ")
print("="*60)

tuned_results = {}

# Decision Tree
print("\nНастройка Decision Tree...")
param_grid_dt = {
    'max_depth': [3, 5, 7, 10, None],
    'min_samples_split': [2, 5, 10]
}
dt = DecisionTreeClassifier(random_state=RANDOM_STATE)
grid_dt = GridSearchCV(dt, param_grid_dt, cv=CV_FOLDS, scoring='accuracy')
grid_dt.fit(X_train, y_train)
print(f"Лучшие параметры: {grid_dt.best_params_}")
print(f"Лучший score: {grid_dt.best_score_:.4f}")
tuned_results['Decision Tree'] = grid_dt.best_estimator_

# SVM
print("\nНастройка SVM...")
param_grid_svm = {
    'C': [0.1, 1, 10],
    'kernel': ['linear', 'rbf', 'poly'],
    'gamma': ['scale', 'auto']
}
svm = SVC(random_state=RANDOM_STATE, probability=True)
grid_svm = GridSearchCV(svm, param_grid_svm, cv=CV_FOLDS, scoring='accuracy')
grid_svm.fit(X_train, y_train)
print(f"Лучшие параметры: {grid_svm.best_params_}")
print(f"Лучший score: {grid_svm.best_score_:.4f}")
tuned_results['SVM'] = grid_svm.best_estimator_

# KNN
print("\nНастройка KNN...")
param_grid_knn = {
    'n_neighbors': [3, 5, 7, 9, 11],
    'weights': ['uniform', 'distance'],
    'metric': ['euclidean', 'manhattan']
}
knn = KNeighborsClassifier()
grid_knn = GridSearchCV(knn, param_grid_knn, cv=CV_FOLDS, scoring='accuracy')
grid_knn.fit(X_train, y_train)
print(f"Лучшие параметры: {grid_knn.best_params_}")
print(f"Лучший score: {grid_knn.best_score_:.4f}")
tuned_results['KNN'] = grid_knn.best_estimator_


In [ ]:
# Оценка настроенных классификаторов
print("\n" + "="*60)
print("ОЦЕНКА НАСТРОЕННЫХ КЛАССИФИКАТОРОВ")
print("="*60)
for name, clf in tuned_results.items():
    y_pred = clf.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average='weighted')
    recall = recall_score(y_test, y_pred, average='weighted')
    f1 = f1_score(y_test, y_pred, average='weighted')
    print(f"{name}:")
    print(f"  Accuracy: {accuracy:.4f}")
    print(f"  Precision: {precision:.4f}")
    print(f"  Recall: {recall:.4f}")
    print(f"  F1-Score: {f1:.4f}")
    
    # Обновление результатов
    if name in results:
        results[name]['accuracy'] = accuracy
        results[name]['precision'] = precision
        results[name]['recall'] = recall
        results[name]['f1'] = f1
        results[name]['y_pred'] = y_pred


## Нейронная сеть на TensorFlow

Нейронная сеть представляет собой вычислительную модель, вдохновленную биологическими нейронными сетями. В данной работе была реализована полносвязная нейронная сеть (feedforward neural network) с использованием библиотеки TensorFlow и Keras. Архитектура сети включает входной слой, один или несколько скрытых слоев с функцией активации ReLU, и выходной слой с функцией активации softmax для многоклассовой классификации.

Для предотвращения переобучения в сеть были добавлены слои dropout, которые случайным образом отключают часть нейронов во время обучения. В качестве оптимизатора использовался Adam, который является адаптивным методом стохастической оптимизации. Функция потерь sparse_categorical_crossentropy была выбрана для многоклассовой классификации с целочисленными метками классов.

Процесс обучения нейронной сети контролировался с помощью TensorBoard, который позволяет визуализировать метрики обучения и валидации в реальном времени. Также был использован механизм ранней остановки (early stopping), который прекращает обучение, если качество на валидационной выборке перестает улучшаться, что помогает предотвратить переобучение.


In [ ]:
# Построение нейронной сети
def build_neural_network(input_dim, num_classes, hidden_layers=[64, 32], learning_rate=0.001):
    """Построение нейронной сети"""
    model = keras.Sequential()
    
    # Входной слой
    model.add(layers.Dense(hidden_layers[0], activation='relu', input_dim=input_dim))
    model.add(layers.Dropout(0.2))
    
    # Скрытые слои
    for units in hidden_layers[1:]:
        model.add(layers.Dense(units, activation='relu'))
        model.add(layers.Dropout(0.2))
    
    # Выходной слой
    model.add(layers.Dense(num_classes, activation='softmax'))
    
    # Компиляция
    optimizer = keras.optimizers.Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer,
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    
    return model

# Обучение нейронной сети
print("\n" + "="*60)
print("ОБУЧЕНИЕ НЕЙРОННОЙ СЕТИ")
print("="*60)

num_classes = len(le.classes_)
model = build_neural_network(X_train.shape[1], num_classes)
print("\nАрхитектура модели:")
model.summary()


In [ ]:
# Callbacks
callbacks = [
    keras.callbacks.TensorBoard(log_dir=TENSORBOARD_DIR, histogram_freq=1),
    keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
]

# Обучение
history = model.fit(X_train, y_train,
                   epochs=EPOCHS,
                   batch_size=BATCH_SIZE,
                   validation_split=VALIDATION_SPLIT,
                   callbacks=callbacks,
                   verbose=1)

# Оценка
test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=0)
print(f"\nTest Accuracy: {test_accuracy:.4f}")
print(f"Test Loss: {test_loss:.4f}")


In [ ]:
# Предсказания и метрики для нейронной сети
y_pred_proba_nn = model.predict(X_test)
y_pred_nn = np.argmax(y_pred_proba_nn, axis=1)

# Метрики
accuracy_nn = accuracy_score(y_test, y_pred_nn)
precision_nn = precision_score(y_test, y_pred_nn, average='weighted')
recall_nn = recall_score(y_test, y_pred_nn, average='weighted')
f1_nn = f1_score(y_test, y_pred_nn, average='weighted')

try:
    roc_auc_nn = roc_auc_score(y_test, y_pred_proba_nn, multi_class='ovr', average='weighted')
except:
    roc_auc_nn = 0.0

nn_results = {
    'model': model,
    'accuracy': accuracy_nn,
    'precision': precision_nn,
    'recall': recall_nn,
    'f1': f1_nn,
    'roc_auc': roc_auc_nn,
    'history': history,
    'y_pred': y_pred_nn,
    'y_pred_proba': y_pred_proba_nn
}

print(f"\nМетрики нейронной сети:")
print(f"  Accuracy: {accuracy_nn:.4f}")
print(f"  Precision: {precision_nn:.4f}")
print(f"  Recall: {recall_nn:.4f}")
print(f"  F1-Score: {f1_nn:.4f}")
print(f"  ROC-AUC: {roc_auc_nn:.4f}")


In [ ]:
# Визуализация истории обучения
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

axes[0].plot(history.history['accuracy'], label='Train Accuracy', linewidth=2)
axes[0].plot(history.history['val_accuracy'], label='Val Accuracy', linewidth=2)
axes[0].set_xlabel('Epoch', fontsize=12)
axes[0].set_ylabel('Accuracy', fontsize=12)
axes[0].set_title('Model Accuracy', fontsize=14)
axes[0].legend(fontsize=10)
axes[0].grid(True, alpha=0.3)

axes[1].plot(history.history['loss'], label='Train Loss', linewidth=2)
axes[1].plot(history.history['val_loss'], label='Val Loss', linewidth=2)
axes[1].set_xlabel('Epoch', fontsize=12)
axes[1].set_ylabel('Loss', fontsize=12)
axes[1].set_title('Model Loss', fontsize=14)
axes[1].legend(fontsize=10)
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
photos_path = os.path.join(photos_dir, 'neural_network_training.png')
plt.savefig(photos_path, dpi=DPI, bbox_inches='tight')
plt.show()


## Сравнительный анализ

Для проведения сравнительного анализа всех рассмотренных методов классификации были рассчитаны метрики качества для каждого метода: точность (accuracy), прецизионность (precision), полнота (recall), F1-мера и площадь под ROC-кривой (AUC-ROC). Эти метрики позволяют оценить различные аспекты качества классификации и выбрать наиболее подходящий метод для данного датасета.

Точность показывает долю правильно классифицированных объектов от общего числа объектов. Прецизионность показывает долю правильно классифицированных положительных примеров среди всех примеров, классифицированных как положительные. Полнота показывает долю верно найденных положительных примеров среди всех положительных примеров. F1-мера является гармоническим средним прецизионности и полноты и позволяет сбалансировать эти две метрики. AUC-ROC показывает способность модели различать классы и является особенно полезной метрикой для несбалансированных датасетов.


In [ ]:
# Сравнение результатов всех методов
comparison = pd.DataFrame({
    'Метод': [],
    'Accuracy': [],
    'Precision': [],
    'Recall': [],
    'F1-Score': [],
    'ROC-AUC': []
})

for name, res in results.items():
    comparison = pd.concat([comparison, pd.DataFrame({
        'Метод': [name],
        'Accuracy': [res['accuracy']],
        'Precision': [res['precision']],
        'Recall': [res['recall']],
        'F1-Score': [res['f1']],
        'ROC-AUC': [res['roc_auc']]
    })], ignore_index=True)

# Добавление нейронной сети
comparison = pd.concat([comparison, pd.DataFrame({
    'Метод': ['Neural Network'],
    'Accuracy': [nn_results['accuracy']],
    'Precision': [nn_results['precision']],
    'Recall': [nn_results['recall']],
    'F1-Score': [nn_results['f1']],
    'ROC-AUC': [nn_results['roc_auc']]
})], ignore_index=True)

print("="*60)
print("СРАВНЕНИЕ РЕЗУЛЬТАТОВ")
print("="*60)
comparison


In [ ]:
# Визуализация сравнения метрик
fig, axes = plt.subplots(2, 3, figsize=(18, 10))
axes = axes.flatten()

metrics = ['Accuracy', 'Precision', 'Recall', 'F1-Score', 'ROC-AUC']
for i, metric in enumerate(metrics):
    axes[i].barh(comparison['Метод'], comparison[metric], color='steelblue', alpha=0.8)
    axes[i].set_xlabel(metric, fontsize=12)
    axes[i].set_title(f'Сравнение {metric}', fontsize=14)
    axes[i].grid(True, alpha=0.3, axis='x')
    axes[i].tick_params(axis='y', labelsize=9)

# Общая визуализация
comparison_melted = comparison.melt(id_vars='Метод', value_vars=metrics, 
                                    var_name='Метрика', value_name='Значение')
sns.barplot(data=comparison_melted, x='Метод', y='Значение', hue='Метрика', ax=axes[5])
axes[5].set_title('Все метрики', fontsize=14)
axes[5].tick_params(axis='x', rotation=45)
axes[5].legend(bbox_to_anchor=(1.05, 1), loc='upper left', fontsize=9)

plt.tight_layout()
photos_path = os.path.join(photos_dir, 'comparison.png')
plt.savefig(photos_path, dpi=DPI, bbox_inches='tight')
plt.show()


In [ ]:
# Визуализация матриц ошибок
n_classifiers = len(results)
fig, axes = plt.subplots(2, (n_classifiers + 1) // 2, figsize=(16, 10))
axes = axes.flatten()

for idx, (name, res) in enumerate(results.items()):
    cm = confusion_matrix(y_test, res['y_pred'])
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', ax=axes[idx],
               xticklabels=le.classes_, yticklabels=le.classes_)
    axes[idx].set_title(f'{name}', fontsize=12)
    axes[idx].set_ylabel('Истинный класс', fontsize=10)
    axes[idx].set_xlabel('Предсказанный класс', fontsize=10)

# Матрица ошибок для нейронной сети
cm_nn = confusion_matrix(y_test, y_pred_nn)
sns.heatmap(cm_nn, annot=True, fmt='d', cmap='Blues', ax=axes[len(results)],
           xticklabels=le.classes_, yticklabels=le.classes_)
axes[len(results)].set_title('Neural Network', fontsize=12)
axes[len(results)].set_ylabel('Истинный класс', fontsize=10)
axes[len(results)].set_xlabel('Предсказанный класс', fontsize=10)

plt.tight_layout()
photos_path = os.path.join(photos_dir, 'confusion_matrices.png')
plt.savefig(photos_path, dpi=DPI, bbox_inches='tight')
plt.show()


## Заключение

В ходе выполнения лабораторной работы был проведен полный анализ различных методов классификации на датасете Iris. Были реализованы и протестированы пять методов классификации из библиотеки Scikit-Learn: наивный Байесовский классификатор (в четырех вариантах), деревья решений, линейный дискриминантный анализ, метод опорных векторов и метод ближайших соседей. Дополнительно была реализована и обучена нейронная сеть с использованием TensorFlow.

Результаты работы показали, что большинство методов демонстрируют высокое качество классификации на данном датасете, что объясняется хорошей разделимостью классов в пространстве признаков. Настройка гиперпараметров позволила улучшить качество некоторых методов, особенно деревьев решений и метода опорных векторов. Нейронная сеть также показала отличные результаты, сопоставимые с лучшими традиционными методами.

Основные выводы работы заключаются в том, что для данного датасета простые методы, такие как линейный дискриминантный анализ и метод опорных векторов, работают столь же эффективно, как и более сложные методы, такие как нейронные сети. Это указывает на то, что для хорошо разделимых данных не всегда необходимо использовать сложные модели. Однако нейронные сети могут быть полезны для более сложных задач с нелинейными зависимостями.

Полученные результаты могут быть использованы для выбора оптимального метода классификации для подобных задач и понимания влияния различных параметров на качество моделей.


## Приложение

Полный листинг программного кода представлен в ячейках данного Jupyter Notebook. Все функции и процедуры, использованные в работе, реализованы с использованием стандартных библиотек Python для машинного обучения и глубокого обучения.

Для просмотра логов TensorBoard выполните команду:
```
tensorboard --logdir=logs
```
